In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('retail_train.csv')
df.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
users, items, interactions = df.user_id.nunique(), df.item_id.nunique(), df.shape[0]

print('# users: ', users)
print('# items: ', items)
print('# interactions: ', interactions)

# users:  2499
# items:  89051
# interactions:  2396804


In [4]:
# разобьём датасет на train и test
test_size_week = 3
df_train = df[df['week_no'] < df['week_no'].max() - test_size_week]
df_test = df[df['week_no'] >= df['week_no'].max() - test_size_week]

In [5]:
# создадим датафрейм с покупками пользователей на тестовом датасете
result = df_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result['actual'] = result['actual'].apply(lambda x: list(x))

In [6]:
# алгоритм случайных рекомендаций
def random_recommendation(items, n=5):
    """Случайные рекомендации"""
        
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [7]:
%%time

items = df.item_id.unique()

# добавим к сводному датафрейму предсказания алгоритма случаных рекомендаций
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

CPU times: user 2.42 s, sys: 4.18 ms, total: 2.43 s
Wall time: 2.43 s


In [8]:
# алгоритм популярных рекомендаций
def popularity_recommendation(data, n=5):
    """Топ-n популярных товаров"""
    
    popular = data.groupby('item_id')['sales_value'].sum().reset_index()
    popular.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popular.head(n).item_id
    
    return recs.tolist()

In [9]:
%%time

popular_recs = popularity_recommendation(df_train, n=5)

result['popularity_recommendation'] = result['user_id'].apply(lambda x: popular_recs)

CPU times: user 69.9 ms, sys: 106 µs, total: 70 ms
Wall time: 69.6 ms


In [10]:
result.head(2)

,user_id,actual,random_recommendation,popularity_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[833155, 986694, 15483407, 1055425, 8119301]","[6534178, 6533889, 1029743, 6534166, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6944607, 905136, 911025, 1095863, 125788]","[6534178, 6533889, 1029743, 6534166, 1082185]"


### Task 0

На вебинаре мы использовали товар 999999 - что это за товар?

Зачем он нужен?

Используя этот товар мы смещаем качество рекомендаций. В какую сторону?

Можно ли удалить этот товар?

Уберите этот товар и сравните с качеством на семинаре.

******

Товар 999999 объединяет в себе все товары, не вошедшие в топ 5к популярных.

Данный товар необходим для того, чтобы мы не потеряли пользователей.

In [11]:
popularity = df_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

In [12]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
df_train.loc[~df_train['item_id'].isin(top_5000), 'item_id'] = 999999

/home/creep/anaconda3/envs/gb_study/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [14]:
user_item_matrix = pd.pivot_table(df_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 
user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# перенумеруем пользователей и товары
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [16]:
%%time

model_item_item = ItemItemRecommender(K=5, num_threads=4) # K - кол-во билжайших соседей

model_item_item.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 931 ms, sys: 7.13 ms, total: 938 ms
Wall time: 413 ms


In [17]:
%%time

result['itemitem'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_item_item.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 208 ms, sys: 3.6 ms, total: 212 ms
Wall time: 60.7 ms


In [18]:
result.head(2)

,user_id,actual,random_recommendation,popularity_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[833155, 986694, 15483407, 1055425, 8119301]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6944607, 905136, 911025, 1095863, 125788]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]"


Используя товар "999999", мы ухудшаем качество рекомендаций.

Среднее значение precision@5 с товаром "999999" = 0.1369

Среднее значение precision@5 с отфильтрованным товаром "999999" = 0.1512

Среднее значение precision@5 без введения товара "999999" = 0.1540

### Task 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [19]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные взвешенные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    items = np.array(items_weights.iloc[:, 0])
    recs = np.random.choice(items, size=n, replace=False, p=np.array(items_weights.iloc[:, 1]))
    
    return recs.tolist()

In [20]:
weights_1 = df.groupby('item_id')['sales_value'].sum().reset_index()

weights_1.loc[weights_1['sales_value'] != 0, 'sales_value'] = (weights_1['sales_value'] + 1).apply(np.log2)
weights_1['sales_value'] = weights_1['sales_value'] / weights_1['sales_value'].sum()

weights_1.rename(columns={'sales_value': 'log_sales_sum'}, inplace=True)

weights_1.head(2)

,item_id,log_sales_sum
0,25671,0.000012
1,26081,0.000003


In [21]:
weights_1['log_sales_sum'].sum()

1.0000000000000002

In [22]:
%%time

items = df.item_id.unique()

# добавим к сводному датафрейму предсказания алгоритма случаных рекомендаций
result['w_random_recommendation_1'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weights_1, n=5))

CPU times: user 2.15 s, sys: 4.35 ms, total: 2.16 s
Wall time: 2.16 s


In [23]:
weights_2 = df.groupby('item_id')['quantity'].sum().reset_index()

weights_2.loc[weights_2['quantity'] != 0, 'quantity'] = (weights_2['quantity'] + 1).apply(np.log2)
weights_2['quantity'] = weights_2['quantity'] / weights_2['quantity'].sum()

weights_2.rename(columns={'quantity': 'log_quantity_sum'}, inplace=True)

weights_2.head(2)

,item_id,log_quantity_sum
0,25671,0.000011
1,26081,0.000004


In [24]:
weights_2['log_quantity_sum'].sum()

1.0

Сделайте предсказания

In [25]:
%%time

items = df.item_id.unique()

# добавим к сводному датафрейму предсказания алгоритма случаных рекомендаций
result['w_random_recommendation_2'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weights_2, n=5))

CPU times: user 2.37 s, sys: 4.18 ms, total: 2.37 s
Wall time: 2.37 s


In [26]:
weights_3 = df.groupby('item_id')['coupon_disc'].sum().reset_index()
weights_3['coupon_disc'] = weights_3['coupon_disc'].apply(np.abs)

weights_3.loc[weights_3['coupon_disc'] != 0, 'coupon_disc'] = (weights_3['coupon_disc'] + 1).apply(np.log2)
weights_3['coupon_disc'] = weights_3['coupon_disc'] / weights_3['coupon_disc'].sum()

weights_3.rename(columns={'coupon_disc': 'log_coupon_disc_sum'}, inplace=True)

weights_3.head(2)

,item_id,log_coupon_disc_sum
0,25671,0.0
1,26081,0.0


In [27]:
weights_3['log_coupon_disc_sum'].sum()

1.0

In [28]:
%%time

items = df.item_id.unique()

# добавим к сводному датафрейму предсказания алгоритма случаных рекомендаций
result['w_random_recommendation_3'] = result['user_id'].apply(lambda x: weighted_random_recommendation(weights_3, n=5))

CPU times: user 2.16 s, sys: 165 µs, total: 2.16 s
Wall time: 2.16 s


### Task 2. Расчёт метрик

Рассчитайте **Precision@5** для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [29]:
def precis_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [30]:
result['precision@5_pop_recs'] = \
result[['popularity_recommendation','actual']].apply(lambda x: precis_at_k(*x), axis=1)

result['precision@5_rand_recs'] = \
result[['random_recommendation','actual']].apply(lambda x: precis_at_k(*x), axis=1)

result['precision@5_wrand_recs_1'] = \
result[['w_random_recommendation_1','actual']].apply(lambda x: precis_at_k(*x), axis=1)

result['precision@5_wrand_recs_2'] = \
result[['w_random_recommendation_2','actual']].apply(lambda x: precis_at_k(*x), axis=1)

result['precision@5_wrand_recs_3'] = \
result[['w_random_recommendation_3','actual']].apply(lambda x: precis_at_k(*x), axis=1)

result['precision@5_itemitem_recs'] = \
result[['itemitem','actual']].apply(lambda x: precis_at_k(*x), axis=1)

In [31]:
mp5_pop_recs = result['precision@5_pop_recs'].mean()
mp5_rand_recs = result['precision@5_rand_recs'].mean()
mp5_wrand_recs_1 = result['precision@5_wrand_recs_1'].mean()
mp5_wrand_recs_2 = result['precision@5_wrand_recs_2'].mean()
mp5_wrand_recs_3 = result['precision@5_wrand_recs_3'].mean()
mp5_itemitem_recs = result['precision@5_itemitem_recs'].mean()

In [32]:
metrics_dict = {}

In [33]:
metrics_dict['Случайные рекомендации'] = mp5_rand_recs * 100
metrics_dict['Случайные взвешенные рекомендации (по сумме проданных товаров)'] = mp5_wrand_recs_1 * 100
metrics_dict['Случайные взвешенные рекомендации (по кол-ву проданных товаров)'] = mp5_wrand_recs_1 * 100
metrics_dict['Случайные взвешенные рекомендации (по размерам скидок)'] = mp5_wrand_recs_3 * 100
metrics_dict['Популярные рекомендации'] = mp5_pop_recs * 100
metrics_dict['Item-Item рекомендации'] = mp5_itemitem_recs * 100

In [34]:
print(f'mean precision@5 для:\n \
Случайные рекомендации = {mp5_rand_recs * 100}%\n \
Случайные взвешенные рекомендации (по сумме проданных товаров) = {mp5_wrand_recs_1 * 100}%\n \
Случайные взвешенные рекомендации (по кол-ву проданных товаров) = {mp5_wrand_recs_1 * 100}%\n \
Случайные взвешенные рекомендации (по размерам скидок) = {mp5_wrand_recs_3 * 100}%\n \
Популярные рекомендации = {mp5_pop_recs * 100}%\n \
Item-Item рекомендации = {mp5_itemitem_recs * 100}%')

mean precision@5 для:
 Случайные рекомендации = 0.039177277179236046%
 Случайные взвешенные рекомендации (по сумме проданных товаров) = 0.1077375122428991%
 Случайные взвешенные рекомендации (по кол-ву проданных товаров) = 0.1077375122428991%
 Случайные взвешенные рекомендации (по размерам скидок) = 0.32321253672869754%
 Популярные рекомендации = 15.523996082272081%
 Item-Item рекомендации = 15.122428991184922%


### Task 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


#### Считаем ItemItem с различным числом соседей

In [35]:
%%time

model_item_item = ItemItemRecommender(K=3, num_threads=4) # K - кол-во билжайших соседей

model_item_item.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 954 ms, sys: 14.9 ms, total: 969 ms
Wall time: 457 ms


In [36]:
%%time

result['itemitem_k3'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_item_item.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 80.6 ms, sys: 51 ms, total: 132 ms
Wall time: 74.6 ms


In [37]:
result['precision@5_itemitemk3_recs'] = \
result[['itemitem_k3','actual']].apply(lambda x: precis_at_k(*x), axis=1)
mp5_itemitemk3_recs = result['precision@5_itemitemk3_recs'].mean()

In [38]:
metrics_dict['Item-Item рекомендации (k=3)'] = mp5_itemitemk3_recs * 100

In [39]:
%%time

model_item_item = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей

model_item_item.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


CPU times: user 1 s, sys: 10.5 ms, total: 1.01 s
Wall time: 481 ms


In [40]:
%%time

result['itemitem_k10'] = result['user_id'].\
    apply(lambda x: [id_to_itemid[rec[0]] for rec in 
                    model_item_item.recommend(userid=userid_to_id[x], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=5, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)])

CPU times: user 147 ms, sys: 67.7 ms, total: 215 ms
Wall time: 79.3 ms


In [41]:
result['precision@5_itemitemk10_recs'] = \
result[['itemitem_k10','actual']].apply(lambda x: precis_at_k(*x), axis=1)
mp5_itemitemk10_recs = result['precision@5_itemitemk10_recs'].mean()

In [42]:
metrics_dict['Item-Item рекомендации (k=10)'] = mp5_itemitemk10_recs * 100

#### Считаем случайные рекомендации только на топ-5000 товаров

In [43]:
%%time

# добавим к сводному датафрейму предсказания алгоритма случаных рекомендаций
result['random_recommendation_top5k'] = result['user_id'].apply(lambda x: random_recommendation(top_5000, n=5))

CPU times: user 648 ms, sys: 217 µs, total: 648 ms
Wall time: 650 ms


In [44]:
result['precision@5_rand_recs_top5k'] = \
result[['random_recommendation_top5k','actual']].apply(lambda x: precis_at_k(*x), axis=1)
mp5_rand_recs_top5k = result['precision@5_rand_recs_top5k'].mean()

In [45]:
metrics_dict['Случайные рекомендации (на топ 5к товаров)'] = mp5_rand_recs_top5k * 100

#### Считаем популярные рекомендации по кластерам

In [46]:
from sklearn.cluster import KMeans

In [47]:
%%time

kmeans = KMeans(n_clusters=10, random_state=42).fit(df)

CPU times: user 38.9 s, sys: 14.4 s, total: 53.3 s
Wall time: 14.6 s


In [48]:
df['user_class'] = kmeans.labels_

In [49]:
df.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,user_class
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,4
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,4


In [50]:
df_train = df[df['week_no'] < df['week_no'].max() - test_size_week]
df_test = df[df['week_no'] >= df['week_no'].max() - test_size_week]

In [51]:
# алгоритм популярных рекомендаций
def popularity_recommendation_clusters(data, clusters, n=5):
    """Топ-n популярных товаров для групп пользователей"""
    
    recs_dict = {}
    for label in np.unique(clusters):
        popular = data.query(f'user_class == {label}').groupby('item_id')['sales_value'].sum().reset_index()
        popular.sort_values('sales_value', ascending=False, inplace=True)
        recs_dict[f'{label}'] = popular.head(n).item_id.tolist()

    dummy_data = data.copy()
    dummy_data['pop_clust_recs'] = dummy_data['user_class']
    dummy_data['pop_clust_recs'] = dummy_data['pop_clust_recs'].apply(lambda x: recs_dict[f'{x}'])
    
    recs = dummy_data['pop_clust_recs'].tolist()
    
    return recs

In [52]:
%%time

popular_recs = popularity_recommendation_clusters(df_train, kmeans.labels_)

result['popularity_recommendation_clusters'] = result['user_id'].apply(lambda x: popular_recs).apply(lambda x: x[0])

CPU times: user 737 ms, sys: 24.3 ms, total: 762 ms
Wall time: 749 ms


In [53]:
result.head(2)

,user_id,actual,random_recommendation,popularity_recommendation,itemitem,w_random_recommendation_1,w_random_recommendation_2,w_random_recommendation_3,precision@5_pop_recs,precision@5_rand_recs,...,precision@5_wrand_recs_2,precision@5_wrand_recs_3,precision@5_itemitem_recs,itemitem_k3,precision@5_itemitemk3_recs,itemitem_k10,precision@5_itemitemk10_recs,random_recommendation_top5k,precision@5_rand_recs_top5k,popularity_recommendation_clusters
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[833155, 986694, 15483407, 1055425, 8119301]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[9883803, 6979697, 2201613, 964356, 10124131]","[1116328, 1081579, 832007, 948811, 984102]","[13133763, 1092221, 5566844, 7128526, 5569071]",0.2,0.0,...,0.0,0.0,0.4,"[1082185, 981760, 995242, 1029743, 840361]",0.4,"[1082185, 981760, 995242, 840361, 1127831]",0.4,"[1091858, 1134398, 945652, 951703, 853172]",0.0,"[6534178, 1029743, 916122, 5569230, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6944607, 905136, 911025, 1095863, 125788]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1030282, 9707007, 938505, 963598, 929421]","[844916, 935834, 9467760, 12159949, 14025442]","[928640, 946477, 9935616, 9677737, 15596278]",0.0,0.0,...,0.0,0.0,0.0,"[1082185, 981760, 1098066, 6534178, 826249]",0.0,"[1082185, 981760, 1098066, 826249, 995242]",0.0,"[938139, 997096, 948832, 835788, 838733]",0.0,"[6534178, 1029743, 916122, 5569230, 1082185]"


In [54]:
result['precision@5_pop_recs_cl'] = \
result[['popularity_recommendation_clusters','actual']].apply(lambda x: precis_at_k(*x), axis=1)
mp5_pop_recs_cl = result['precision@5_pop_recs_cl'].mean()

In [55]:
metrics_dict['Популярные рекомендации по кластерам'] = mp5_pop_recs_cl * 100

In [56]:
pd.DataFrame(list(metrics_dict.items()),
             columns=['Метод', 'Значение, %']).sort_values(by=['Значение, %'], ascending=False)

,Метод,"Значение, %"
6,Item-Item рекомендации (k=3),21.897649
9,Популярные рекомендации по кластерам,18.403526
7,Item-Item рекомендации (k=10),16.601371
4,Популярные рекомендации,15.523996
5,Item-Item рекомендации,15.122429
8,Случайные рекомендации (на топ 5к товаров),0.617042
3,Случайные взвешенные рекомендации (по размерам...,0.323213
1,Случайные взвешенные рекомендации (по сумме пр...,0.107738
2,Случайные взвешенные рекомендации (по кол-ву п...,0.107738
0,Случайные рекомендации,0.039177


### Task 4. Улучшение детерминированных алгоритмов

Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.